## <center>This Model will Attempt to Predict the Type of Bicycle Used from Trip Data</center>

In [1]:
# Eliminating Warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importing Libraries
import pandas as pd
import numpy as np
import os
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import SMOTE
from sklearn.metrics import balanced_accuracy_score

In [3]:
# Set up data path
data_path = 'data'

In [4]:
# Load base data to dataframe
all_data_file = 'all_data.csv'
all_df = pd.read_csv(os.path.join(data_path, all_data_file))
all_df

,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type
0,11/12/16,To Essex Market,3.43,NaN,29.0,0.48,7.10,Citibike
1,8/23/16,Citibike to Work,1.46,NaN,15.0,0.25,5.84,Citibike
2,9/12/20,Wards Island x 3,22.91,2.0,35.0,2.58,8.87,Specialized
3,9/4/20,To Astor Wines,2.92,NaN,25.0,0.42,7.01,Citibike
4,4/7/20,CP BL x 2,13.98,1.0,43.0,1.72,8.14,Specialized
...,...,...,...,...,...,...,...,...
356,1/17/21,Dn ES Up LWS,15.41,2.0,2.0,2.03,7.58,Specialized
357,8/4/16,Citibike to Work,1.40,NaN,11.0,0.18,7.64,Citibike
358,1/13/17,Citibike from Work,1.01,NaN,8.0,0.13,7.58,Citibike
359,6/2/20,From Astoria,5.38,NaN,52.0,0.87,6.21,Specialized


In [5]:
# Change NaN values to 0 - only occurs on hours column so this is accurate
all_df['Hours'] = all_df['Hours'].fillna(0)
all_df

,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type
0,11/12/16,To Essex Market,3.43,0.0,29.0,0.48,7.10,Citibike
1,8/23/16,Citibike to Work,1.46,0.0,15.0,0.25,5.84,Citibike
2,9/12/20,Wards Island x 3,22.91,2.0,35.0,2.58,8.87,Specialized
3,9/4/20,To Astor Wines,2.92,0.0,25.0,0.42,7.01,Citibike
4,4/7/20,CP BL x 2,13.98,1.0,43.0,1.72,8.14,Specialized
...,...,...,...,...,...,...,...,...
356,1/17/21,Dn ES Up LWS,15.41,2.0,2.0,2.03,7.58,Specialized
357,8/4/16,Citibike to Work,1.40,0.0,11.0,0.18,7.64,Citibike
358,1/13/17,Citibike from Work,1.01,0.0,8.0,0.13,7.58,Citibike
359,6/2/20,From Astoria,5.38,0.0,52.0,0.87,6.21,Specialized


In [6]:
# Turning Index into Orig_Index Column
all_df.reset_index(inplace=True)
all_df = all_df.rename(columns = {'index': 'Orig_Index'})
all_df

,Orig_Index,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type
0,0,11/12/16,To Essex Market,3.43,0.0,29.0,0.48,7.10,Citibike
1,1,8/23/16,Citibike to Work,1.46,0.0,15.0,0.25,5.84,Citibike
2,2,9/12/20,Wards Island x 3,22.91,2.0,35.0,2.58,8.87,Specialized
3,3,9/4/20,To Astor Wines,2.92,0.0,25.0,0.42,7.01,Citibike
4,4,4/7/20,CP BL x 2,13.98,1.0,43.0,1.72,8.14,Specialized
...,...,...,...,...,...,...,...,...,...
356,356,1/17/21,Dn ES Up LWS,15.41,2.0,2.0,2.03,7.58,Specialized
357,357,8/4/16,Citibike to Work,1.40,0.0,11.0,0.18,7.64,Citibike
358,358,1/13/17,Citibike from Work,1.01,0.0,8.0,0.13,7.58,Citibike
359,359,6/2/20,From Astoria,5.38,0.0,52.0,0.87,6.21,Specialized


In [7]:
# Defining fields to use
columns = ['Miles', 'Duration', 'Speed','Type']
# target = ['Type']

In [8]:
# Create Dataframe with desired columns
bike_df = all_df.loc[:,columns].copy()
bike_df

,Miles,Duration,Speed,Type
0,3.43,0.48,7.10,Citibike
1,1.46,0.25,5.84,Citibike
2,22.91,2.58,8.87,Specialized
3,2.92,0.42,7.01,Citibike
4,13.98,1.72,8.14,Specialized
...,...,...,...,...
356,15.41,2.03,7.58,Specialized
357,1.40,0.18,7.64,Citibike
358,1.01,0.13,7.58,Citibike
359,5.38,0.87,6.21,Specialized


In [9]:
# Create Dataframe with desired columns
# Demonstrating different way to do it
# bike_df = all_df[['Miles', 'Duration', 'Speed','Type']]
# bike_df

In [10]:
# Splitting out Features and Target
y = bike_df["Type"]
X = bike_df.drop(columns="Type")

In [11]:
# Splitting out Training and Testing data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [12]:
# Checking training data shape
X_train.shape

(270, 3)

In [13]:
# Checking testing data shape
X_test.shape

(91, 3)

In [14]:
# Creating the Logistic Regression Model
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs', max_iter=200, random_state=1)

In [15]:
# Training the Model
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=200, random_state=1)

In [16]:
# Create Predictions
y_pred = classifier.predict(X_test)

In [17]:
# Get accuracy score
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.978021978021978


In [18]:
# Convert Predictions to DataFrame
pred_df = pd.DataFrame(y_pred)
pred_df.reset_index()
pred_df = pred_df.rename(columns = {0:'Prediction'})
pred_df

,Prediction
0,Citibike
1,Specialized
2,Citibike
3,Citibike
4,Citibike
...,...
86,Specialized
87,Specialized
88,Specialized
89,Specialized


In [19]:
# Convert Tests to DataFrame
test_df = pd.DataFrame(y_test)
test_df

,Type
219,Citibike
192,Specialized
181,Citibike
201,Citibike
313,Citibike
...,...
74,Specialized
323,Specialized
121,Specialized
70,Specialized


In [20]:
# Making index into a column
test_df.reset_index(inplace=True)
test_df = test_df.rename(columns = {'index': 'Orig_Index'})
test_df

,Orig_Index,Type
0,219,Citibike
1,192,Specialized
2,181,Citibike
3,201,Citibike
4,313,Citibike
...,...,...
86,74,Specialized
87,323,Specialized
88,121,Specialized
89,70,Specialized


In [21]:
# Combining the new test and predition dataframes horizontally for comparison
test_pred_df = pd.concat([test_df, pred_df], axis = 1)
test_pred_df

,Orig_Index,Type,Prediction
0,219,Citibike,Citibike
1,192,Specialized,Specialized
2,181,Citibike,Citibike
3,201,Citibike,Citibike
4,313,Citibike,Citibike
...,...,...,...
86,74,Specialized,Specialized
87,323,Specialized,Specialized
88,121,Specialized,Specialized
89,70,Specialized,Specialized


In [22]:
# Identify records where prediction is wrong
pred_errs_df = test_pred_df.loc[test_pred_df['Type'] != test_pred_df['Prediction']]
pred_errs_df

,Orig_Index,Type,Prediction
13,47,Specialized,Citibike
49,94,Citibike,Specialized


In [23]:
# Merging pred_errs_df with all_df to see details of errors
err_details_df = pd.merge(pred_errs_df, all_df, on=["Orig_Index", "Orig_Index"])
err_details_df

,Orig_Index,Type_x,Prediction,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type_y
0,47,Specialized,Citibike,7/28/19,Roosevelt Island,4.25,0.0,39.0,0.65,6.54,Specialized
1,94,Citibike,Specialized,6/17/18,Work to Village,4.69,0.0,40.0,0.67,7.04,Citibike


In [24]:
# Cleaning up err_details_df to eliminate redundancy and improve readability
err_details_df.drop(columns='Type_y', inplace=True)
err_details_df.rename(columns = {'Type_x' : 'Type'}, inplace=True)
err_details_df

,Orig_Index,Type,Prediction,Date,Destination,Miles,Hours,Minutes,Duration,Speed
0,47,Specialized,Citibike,7/28/19,Roosevelt Island,4.25,0.0,39.0,0.65,6.54
1,94,Citibike,Specialized,6/17/18,Work to Village,4.69,0.0,40.0,0.67,7.04
